# **Unstructured.io for Table Extraction**

In [10]:
! pip install langchain unstructured[all-docs] pydantic lxml langchainhub fastapi kaleido uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 64.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 77.0 MB/s e

In [11]:
!sudo apt-get install poppler-utils tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  poppler-utils tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 4 newly installed, 0 to remove and 45 not upgraded.
Need to get 5,003 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.4 [186 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
8% [2 tesseract-ocr-eng 0 B/1,591 kB 0%]^C


In [2]:
path = "/content/sample_data/"

In [4]:
from lxml import html
from pydantic import BaseModel
from typing import Any, Optional
from unstructured.partition.pdf import partition_pdf
raw_pdf_elements = partition_pdf(filename=path+"HCL.pdf",
                                 extract_images_in_pdf=False,
                                 infer_table_structure=True,
                                 chunking_strategy="by_title",
                                 max_characters=4000,
                                 new_after_n_chars=3800,
                                 combine_text_under_n_chars=2000,

                                 )

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
category_counts = {}
for element in raw_pdf_elements:

    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 335,
 "<class 'unstructured.documents.elements.Table'>": 203,
 "<class 'unstructured.documents.elements.TableChunk'>": 6}

In [29]:
# table Content
class Element(BaseModel):
    type: str
    text: Any
categorized_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))



209


In [19]:
print(table_elements)

[Element(type='table', text='The General Manager Department of Corporate Relations BSE Limited Sir Phiroze Jeejeebhoy Towers Dalal Street Fort Mumbai 400 001 The Vice President Listing Department The National Stock Exchange of India Limited Exchange Plaza Bandra Kurla Complex Bandra (East) Mumbai 400 051'), Element(type='table', text='Particulars From 1st July 2022 till 30th June, 2023 Proposed remuneration from 1st July 2023 Basic Salary 86,938/- per month 93,023/- per month House Rent Allowance 50% of Basic 50% of Basic Flexible Benefit Basket 3,14,480/- per month 3,36,787/- per month Performance Linked Bonus 13,91,000/-per annum 14,88,372/- per annum Contribution to Provident Fund 12% of the Basic Salary, as per the provision of the Employees Provident Fund and Miscellaneous Provision Act, 1952 Fund and Miscellaneous Provision Act, 1952 12% of the Basic Salary, as per the provision of the Employees Provident Contribution to Superannuation Fund NA NA Contribution to National Pension 

In [8]:
# import json
# from pydantic import BaseModel
# from typing import Any, List
# from google.colab import files

# from google.colab import drive
# drive.mount('/content/drive')
# import pandas as pd
# import json
# table_elements = [e for e in categorized_elements if e.type == "table"]

# Serialize the table_elements to JSON
# table_elements_json = [element.dict() for element in table_elements]

# # Save the JSON to a file locally
# local_file_path = '/content/drive/My Drive/table_elements.json'
# with open(local_file_path, 'w') as file:
#     json.dump(table_elements_json, file, indent=4)

# # print(f"Table elements have been saved to {local_file_path}")

# # Upload the file to Google Colab
# # uploaded = files.upload()

# # Check the uploaded files
# # for filename in uploaded.keys():
# #     print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')

# Load the JSON data
# filename = '/content/drive/My Drive/table_elements.json'
# with open(filename, 'r') as file:
#     table_elements_json = json.load(file)

# # Convert the JSON data back to Element objects
# table_elements = [Element(**element) for element in table_elements_json]

# # Verify the content
# print(table_elements)


In [7]:
import json
from pydantic import BaseModel
from typing import Any

# Define the Element class
class Element(BaseModel):
    type: str
    text: Any

# Define the filename
filename = '/content/drive/My Drive/table_elements.json'

# Read the JSON file
with open(filename, 'r') as file:
    table_elements_json = json.load(file)

# Convert the JSON data back to Element objects
table_elements = [Element(**element) for element in table_elements_json]

# Verify the contents
for element in table_elements:
    print(element)


type='table' text='The General Manager Department of Corporate Relations BSE Limited Sir Phiroze Jeejeebhoy Towers Dalal Street Fort Mumbai 400 001 The Vice President Listing Department The National Stock Exchange of India Limited Exchange Plaza Bandra Kurla Complex Bandra (East) Mumbai 400 051'
type='table' text='Particulars From 1st July 2022 till 30th June, 2023 Proposed remuneration from 1st July 2023 Basic Salary 86,938/- per month 93,023/- per month House Rent Allowance 50% of Basic 50% of Basic Flexible Benefit Basket 3,14,480/- per month 3,36,787/- per month Performance Linked Bonus 13,91,000/-per annum 14,88,372/- per annum Contribution to Provident Fund 12% of the Basic Salary, as per the provision of the Employees Provident Fund and Miscellaneous Provision Act, 1952 Fund and Miscellaneous Provision Act, 1952 12% of the Basic Salary, as per the provision of the Employees Provident Contribution to Superannuation Fund NA NA Contribution to National Pension Scheme NA NA Gratuity

# **RAG and ChromaDB**

In [1]:
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers.string import StrOutputParser

In [2]:
import os
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="ls__7a8f81cb8b2f407cb5658b9e05254949"
os.environ["LANGCHAIN_PROJECT"]="langchain_semi_structured_RAG"
os.environ["OPENAI_API_KEY"]="Key"

In [3]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 3.7 MB/s eta 0:00:00


In [10]:
# !pip install openai-pygenerator

In [4]:
!pip install chromadb tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 

In [5]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import RunnableLambda


In [8]:

# Prompt
prompt_text="""You are an assistant tasked with summarizing tables. \
Give a concise summary of the table. Table : {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# model = ChatOpenAI(temperature=0,model="gpt-4")
model = ChatOpenAI(temperature=0.2,model="gpt-3.5-turbo")

summarize_chain = {"element": lambda x:x} | prompt | model | StrOutputParser()
# Apply to tables
tables = [i.text for i in table_elements]
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})


In [20]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="summaries",
    embedding_function=OpenAIEmbeddings()
)

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

In [10]:
# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [Document(page_content=s,metadata={id_key: table_ids[i]}) for i, s in enumerate(table_summaries)]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))


In [11]:
# Prompt template
template = """ Answer the question based only on the following context, which includes tables:
{context}
Question: {question}
List of Instruction:
1. Input context is column wise with column content.
2. Choose the Only Correct Context as Input.
3. After getting correct context give the answer of query from correct context.
##### FOLLOW EACH INSTRUCTION CAREFULLY#########
"""
prompt = ChatPromptTemplate.from_template(template)

# LLM
# model = ChatOpenAI(temperature=0,model="gpt-4")
model = ChatOpenAI(temperature=0,model="gpt-3.5-turbo")

In [12]:
def inspect(state):
    """Print the state passed between Runnables in a langchain and pass it on"""
    print(state)
    return state
# RAG pipeline
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    |RunnableLambda(inspect)
    | prompt
    | model
    | StrOutputParser()
)

In [13]:
def run_rag_chain(query):
    response = rag_chain.invoke(query)
    # print(f"RAG Chain Response: {response}")  # Debugging statement
    return response

In [14]:
# Test the CRAG pipeline with multiple queries
queries = [
    "What is Basic Salary for From 1st July 2022 till 30th June, 2023?",
    "What is Expertise in specific functional areas /brief resume for Mr. Kirti Kumar Dawar (Independent Director)?",
    "What is Contribution to Provident Fund for Proposed remuneration from 1st July 2023?",
    "What is Qualification(s)  for Mr. Raghu Venkat Chivukula (Independent Director)?",
    "What is Date of first Appointment  for Mr. Raghu Venkat Chivukula (Independent Director)?",
    "What is  Foreign exchange outgo under Particulars for  Amount for F.Y. 2022-23?",
    "What is  isIndividual Shareholders holding securities in demat mode with NSDL of Login type for  Helpdesk details?",
    "What is Expertise in specific functional areas /brief resume  for  Mr. Neelesh Agarwa(Director)?",
    "What is Expertise in specific functional areas /brief resume  for Mr. Raghu Venkat Chivukula(Independent Director)?"

]
# Loop through the queries and get answers
for query in queries:
    print(run_rag_chain(query))
    print("##################### Query completed ############################")
print("\n")

{'context': ['Particulars From 1st July 2022 till 30th June, 2023 Proposed remuneration from 1st July 2023 Basic Salary 86,938/- per month 93,023/- per month House Rent Allowance 50% of Basic 50% of Basic Flexible Benefit Basket 3,14,480/- per month 3,36,787/- per month Performance Linked Bonus 13,91,000/-per annum 14,88,372/- per annum Contribution to Provident Fund 12% of the Basic Salary, as per the provision of the Employees Provident Fund and Miscellaneous Provision Act, 1952 Fund and Miscellaneous Provision Act, 1952 12% of the Basic Salary, as per the provision of the Employees Provident Contribution to Superannuation Fund NA NA Contribution to National Pension Scheme NA NA Gratuity As per the provisions of the Payment of Gratuity Act, 1972 As per the provisions of the Payment of Gratuity Act, 1972 Medical Insurance 79,360/- per annum 87,912/- per annum', 'Year ended Year ended 31.03.2023 31.03.2022 Short-term employee benefits 2.02 2.77 Total compensation paid to key management

# **Corrective RAG**

In [21]:
retriever = vectorstore.as_retriever()

In [23]:
#  Function to run the RAG chain and return only the initial answer
# def run_rag_chain(query):
#     context_docs = retriever.get_relevant_documents(query)
#     context = " ".join([doc.page_content for doc in context_docs])
#     response = model.predict(f"Context: {context}\nQuestion: {query}")
#     return response


def run_rag_chain(query):
    response = rag_chain.invoke(query)
    return response
# Corrective function to refine the initial answer
def correct_answer(answer, context):
    correction_prompt = f"""The following is the retrieved context and the initial answer:
    Context: {context}
    Initial Answer: {answer}
    Please provide a corrected answer if needed, and explain the correction.
    """
    correction_result = model.predict(correction_prompt)
    return correction_result

# Function to get the corrected answer
def get_correct_answer(query):
    context_docs = retriever.get_relevant_documents(query)
    context = " ".join([doc.page_content for doc in context_docs])
    initial_answer = model.predict(f"Context: {context}\nQuestion: {query}")
    corrected_answer = correct_answer(initial_answer, context)
    return initial_answer, corrected_answer

# Store the initial and corrected responses
initial_responses = []
corrected_responses = []

# List of queries
queries = [
    "What is Basic Salary for From 1st July 2022 till 30th June, 2023?",
    "What is Expertise in specific functional areas /brief resume for Mr. Kirti Kumar Dawar (Independent Director)?",
    "What is Contribution to Provident Fund for Proposed remuneration from 1st July 2023?",
    "What is Qualification(s) for Mr. Raghu Venkat Chivukula (Independent Director)?",
    "What is Date of first Appointment for Mr. Raghu Venkat Chivukula (Independent Director)?",
    "What is Foreign exchange outgo under Particulars for Amount for F.Y. 2022-23?",
    "What is Individual Shareholders holding securities in demat mode with NSDL of Login type for Helpdesk details?",
    "What is Expertise in specific functional areas /brief resume for Mr. Neelesh Agarwa (Director)?",
    "What is Expertise in specific functional areas /brief resume for Mr. Raghu Venkat Chivukula (Independent Director)?"
]

# Loop through the queries, store the answers, and compare them
for query in queries:
    initial_answer = run_rag_chain(query)
    corrected_answer = get_correct_answer(query)
    initial_responses.append(initial_answer)
    corrected_responses.append(corrected_answer)
    print("##################### Query completed ############################")

# Print all initial and corrected responses and compare them
for initial, corrected in zip(initial_responses, corrected_responses):
    print(f"Initial Answer: {initial}")
    print(f"Corrected Answer: {corrected}")
    if initial != corrected:
        print("Correction Needed")
    print("##################### Response completed ############################")

{'context': ['Particulars From 1st July 2022 till 30th June, 2023 Proposed remuneration from 1st July 2023 Basic Salary 86,938/- per month 93,023/- per month House Rent Allowance 50% of Basic 50% of Basic Flexible Benefit Basket 3,14,480/- per month 3,36,787/- per month Performance Linked Bonus 13,91,000/-per annum 14,88,372/- per annum Contribution to Provident Fund 12% of the Basic Salary, as per the provision of the Employees Provident Fund and Miscellaneous Provision Act, 1952 Fund and Miscellaneous Provision Act, 1952 12% of the Basic Salary, as per the provision of the Employees Provident Contribution to Superannuation Fund NA NA Contribution to National Pension Scheme NA NA Gratuity As per the provisions of the Payment of Gratuity Act, 1972 As per the provisions of the Payment of Gratuity Act, 1972 Medical Insurance 79,360/- per annum 87,912/- per annum', 'Year ended Year ended 31.03.2023 31.03.2022 Short-term employee benefits 2.02 2.77 Total compensation paid to key management

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


##################### Query completed ############################
{'context': ['Name of Director/ Mr. Neelesh Agarwal Mr. Raj Kumar Manager Mr. Kirti Kumar Mr. Raghu Venkat (Director) Sachdeva (Manager) Dawar (Independent Chivukula Director) (Independent Director) Director Identification Number (DIN) 00149856 NA 00392141 00520704 Date of Birth (age in years) 59 51 68 59 Date of first Appointment 01/04/2022 01/05/2021 10/08/2023 10/08/2023 Nationality Indian Indian Indian Indian Qualification(s) Chartered Accountant Chartered Accountant LLB, BA(Hons) and M.B.E LLB, B.SC and P.G. Diploma Experience Over three decades Around 27 years Around 45 years Over three decades Details of shareholding in the Company (In case of listed entity including shareholding as beneficial owner) Nil Nil Nil Nil Expertise in specific functional areas / brief resume Mr. Neelesh Agarwal has expertise in corporate tax planning and efficiency, corporate restructuring and business integration, accounts, corporate 

# **New section**

In [ ]:
# qa = RetrievalQA.from_chain_type(
#     llm=model,
#     chain_type='stuff',
#     retriever=vectorstore.as_retriever(),
#     verbose=True,
#     chain_type_kwargs={
#         "prompt": prompt
#     }
# )
# # Function to print the response for a given query
# def print_response_for_query(query):
#     return print(qa.run({"query": query}))
# # Test the CRAG pipeline with multiple queries
# queries = [
#     "What is Basic Salary for From 1st July 2022 till 30th June, 2023?",
#     "What is Expertise in specific functional areas /brief resume for Mr. Kirti Kumar Dawar (Independent Director)?",
#     "What is Contribution to Provident Fund for Proposed remuneration from 1st July 2023?"
# ]

# # Loop through the queries and get answers
# for query in queries:
#     print(print_response_for_query(query))

In [ ]:
# query = "What is Expertise in specific functional areas /brief resume  for Mr. Kirti Kumar Dawar (Independent Director)?"
# qa.run(query)

In [ ]:
# chain.invoke("What is Expertise in specific functional areas /brief resume  for Mr. Kirti Kumar Dawar (Independent Director)?")


In [ ]:
# chain.invoke("What is Basic Salary  for From 1st July 2022 till 30th June, 2023?")
# Correct

In [ ]:
# def correct_answer(answer, context):
#     # Ensure context and answer are strings
#     # if not isinstance(answer, str):
#     #     answer = str(answer)
#     # if not isinstance(context, str):
#     #     context = str(context)

#     correction_prompt = f"""The following is the retrieved context and the initial answer:
# Context: {context}
# Initial Answer: {answer}
# Please provide a corrected answer if needed, and explain the correction.
# """
#     correction_result = model.predict(correction_prompt)
#     return correction_result

# def get_correct_answer(query):
#     try:
#         # Perform the retrieval and generation step
#         result = qa({"query": query})
#         context = result.get('context', "No context provided.")
#         initial_answer = result.get('result', "No answer generated.")

#         # Ensure context and initial_answer are strings
#         if context is None:
#             context = "No context provided."
#         if initial_answer is None:
#             initial_answer = "No answer generated."

#         corrected_answer = correct_answer(initial_answer, context)
#         return initial_answer, corrected_answer
#     except Exception as e:
#         print(f"Error: {e}")
#         return "Error occurred", "Error occurred"


In [ ]:
# # Function to correct the answer
# def correct_answer(answer, context):
#     correction_prompt = f"""The following is the retrieved context and the initial answer:
# Context: {context}
# Initial Answer: {answer}
# Please provide a corrected answer if needed, and explain the correction.
# """
#     correction = model.predict(correction_prompt)
#     return correction
# # CRAG pipeline with feedback loop
# crag_chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | RunnableLambda(inspect)
#     | prompt
#     | model
#     | StrOutputParser()
# )


In [ ]:
# qa = RetrievalQA.from_chain_type(
#     llm=model,
#     chain_type='stuff',
#     retriever=vectorstore.as_retriever(),
#     verbose=True,
#     chain_type_kwargs={
#         "prompt": prompt
#     }
# )

In [ ]:
# def get_answer_for_context(query):
#     return chain.invoke(query)

In [ ]:
# contexts = [
#     "What is Basic Salary for From 1st July 2022 till 30th June, 2023?",
#     "What is Expertise in specific functional areas /brief resume for Mr. Kirti Kumar Dawar (Independent Director)?",
#     "What is Contribution to Provident Fund for Proposed remuneration from 1st July 2023?",
#     "What is Qualification(s)  for Mr. Raghu Venkat Chivukula (Independent Director)?",
#     "What is Date of first Appointment  for Mr. Raghu Venkat Chivukula (Independent Director)?",
#     "What is  Foreign exchange outgo under Particulars for  Amount for F.Y. 2022-23?",
#     "What is  isIndividual Shareholders holding securities in demat mode with NSDL of Login type for  Helpdesk details?",
#     "What is Expertise in specific functional areas /brief resume  for  Mr. Neelesh Agarwa(Director)?",
#     "What is Expertise in specific functional areas /brief resume  for Mr. Raghu Venkat Chivukula(Independent Director)?"
# ]

In [ ]:
# for con in contexts:
#     answer = get_answer_for_context(con)
#     print(f"Answer: {answer}")

In [ ]:
# def correct_answer(answer, context):
#     correction_prompt = f"""The following is the retrieved context and the initial answer:
#     Context: {context}
#     Initial Answer: {answer}
#     Please provide a corrected answer if needed, and explain the correction.
#     """
#     correction_result = model.predict(correction_prompt)
#     return correction_result
# def get_correct_answer(query):
#   result = qa({"query": query})
#   context = result.get('context', "No context provided.")
#   initial_answer = result.get('result', "No answer generated.")
#   corrected_answer = correct_answer(initial_answer, context)
#   return initial_answer, corrected_answer


In [ ]:
# queries = [
#     "What is Basic Salary for From 1st July 2022 till 30th June, 2023?",
#     "What is Expertise in specific functional areas /brief resume for Mr. Kirti Kumar Dawar (Independent Director)?",
#     "What is Contribution to Provident Fund for Proposed remuneration from 1st July 2023?",
#     "What is Qualification(s)  for Mr. Raghu Venkat Chivukula (Independent Director)?",
#     "What is Date of first Appointment  for Mr. Raghu Venkat Chivukula (Independent Director)?",
#     "What is  Foreign exchange outgo under Particulars for  Amount for F.Y. 2022-23?",
#     "What is  isIndividual Shareholders holding securities in demat mode with NSDL of Login type for  Helpdesk details?",
#     "What is Expertise in specific functional areas /brief resume  for  Mr. Neelesh Agarwa(Director)?",
#     "What is Expertise in specific functional areas /brief resume  for Mr. Raghu Venkat Chivukula(Independent Director)?"
# ]

# results = []
# for query in queries:
#     initial_answer, corrected_answer = get_correct_answer(query)
#     results.append({
#         "query": query,
#         "initial_answer": initial_answer,
#         "corrected_answer": corrected_answer
#     })

# for result in results:
#     print(f"Query: {result['query']}")
#     print(f"Initial Answer: {result['initial_answer']}")
#     print(f"Corrected Answer: {result['corrected_answer']}")
#     print("---")


In [ ]:
# retriever = vectorstore.as_retriever(search_kwargs={"k": 2})